In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

In [ ]:
import logging
import sys
import pandas as pd

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.embeddings.fastembed import FastEmbedEmbedding 
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 512  # Optional: controls how large each text chunk is for embedding

df1 = pd.read_csv('llmdataset.csv')

documents = [Document(text=' '.join(row.values.astype(str))) for _, row in df1.iterrows()]

index = VectorStoreIndex.from_documents(documents)

print("Vector index created.")


In [ ]:
from llama_index.core import PromptTemplate

system_prompt = ("You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the context provided to you.")
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")  
]

llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    stopping_ids=stopping_ids,
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.llm = llm
Settings.chunk_size = 512

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()
def predict(input, history=None):
    response = query_engine.query(input)
    return str(response)


In [ ]:
user_question = input("Ask a question about Noonan Syndrome: ")
response = query_engine.query(user_question)
print(response)


In [ ]:
import gradio as gr
gr.ChatInterface(predict).launch(share=True)